# Advent of Code 2018

In [1]:
import os
import re
import string

from collections import Counter, defaultdict, deque
from datetime import datetime
from itertools import accumulate, chain, combinations, cycle, zip_longest
import operator as op

import networkx as nx
import numpy as np
import pandas as pd
import requests

from scipy.spatial.distance import cdist

%load_ext blackcellmagic
%matplotlib inline

In [2]:
def input_for_day(day):
    file_name = f"day{day:0>2}_input"
    if os.path.exists(file_name):
        return os.path.abspath(file_name)
    with open("session", "r") as session_file:
        response = requests.get(
            f"https://adventofcode.com/2018/day/{day}/input",
            cookies={"session": session_file.read()},
        )
    with open(file_name, "w") as input_file:
        print(response.text.strip(), file=input_file)
    return os.path.abspath(file_name)

## Day 1
### Part One
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

In [3]:
%%time
with open(input_for_day(1), "r") as input_file:
    print(f"Answer: {sum(map(int, input_file))}")

Answer: 538
CPU times: user 1.95 ms, sys: 636 µs, total: 2.58 ms
Wall time: 27.6 ms


### Part Two
What is the first frequency your device reaches twice?

In [4]:
%%time
freq_set = {0}
with open(input_for_day(1), "r") as input_file:
    for freq in accumulate(cycle(int(line) for line in input_file)):
        if freq in freq_set or freq_set.add(freq):
            print(f"Answer: {freq}")
            break

Answer: 77271
CPU times: user 33.8 ms, sys: 11.3 ms, total: 45.1 ms
Wall time: 58.6 ms


## Day 2
### Part One
What is the checksum for your list of box IDs?

In [5]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = [Counter(n.strip()).values() for n in input_file]
checksum = sum(2 in c for c in counters) * sum(3 in c for c in counters)
print(f"Answer: {checksum}")

Answer: 5976
CPU times: user 15 ms, sys: 1.61 ms, total: 16.6 ms
Wall time: 39.8 ms


### Part Two
What letters are common between the two correct box IDs?

In [6]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = {x: Counter(x).values() for x in (n.strip() for n in input_file)}
for x, y in combinations(
    (n for n, c in counters.items() if any(s in c for s in (2, 3))), 2
):
    if sum(a != b for a, b in zip(x, y)) == 1:
        print(f"Answer: {''.join(a for a, b in zip(x, y) if a == b)}")
        break

Answer: xretqmmonskvzupalfiwhcfdb
CPU times: user 16 ms, sys: 8.75 ms, total: 24.7 ms
Wall time: 41.7 ms


## Day 3
### Part One
How many square inches of fabric are within two or more claims?

In [7]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    for claim in input_file:
        _, x, y, w, h = map(int, expr.match(claim).groups())
        fabric[y : y + h, x : x + w] += 1
    print(f"Answer: {np.sum(fabric > 1)}")

Answer: 117948
CPU times: user 19.5 ms, sys: 9.89 ms, total: 29.3 ms
Wall time: 56.7 ms


### Part Two
What is the ID of the only claim that doesn't overlap?

In [8]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    claims = [list(map(int, expr.match(claim).groups())) for claim in input_file]
for claim in claims:
    _, x, y, w, h = claim
    fabric[y : y + h, x : x + w] += 1
for claim in claims:
    claim_id, x, y, w, h = claim
    if np.all(fabric[y : y + h, x : x + w] == 1):
        print(f"Answer: {claim_id}")
        break

Answer: 567
CPU times: user 33.3 ms, sys: 841 µs, total: 34.1 ms
Wall time: 58.5 ms


## Day 4
### Part One
What is the ID of the guard you chose multiplied by the minute you chose?

In [9]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

_, guard = max((sum(c.values()), g) for g, c in guards.items())

print(f"Answer: {guard * guards[guard].most_common()[0][0]}")

Answer: 4716
CPU times: user 32.7 ms, sys: 640 µs, total: 33.4 ms
Wall time: 65.6 ms


### Part Two
What is the ID of the guard you chose multiplied by the minute you chose? 

In [10]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

(_, minute), guard = max((c.most_common()[0][::-1], g) for g, c in guards.items())

print(f"Answer: {guard * minute}")

Answer: 117061
CPU times: user 31.4 ms, sys: 188 µs, total: 31.6 ms
Wall time: 60.5 ms


## Day 5
### Part One
How many units remain after fully reacting the polymer you scanned?

In [11]:
%%time
def collapse(s):
    p = ["_"]
    for u in s:
        v = p[-1]
        p.pop() if v != u and v.lower() == u.lower() else p.append(u)

    return len(p) - 1


with open(input_for_day(5), "r") as input_file:
    print(f"Answer: {collapse(input_file.read().strip())}")

Answer: 11252
CPU times: user 23.2 ms, sys: 617 µs, total: 23.9 ms
Wall time: 50.9 ms


### Part Two
What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [12]:
%%time
def collapse(s):
    p = ["_"]
    for u in s:
        v = p[-1]
        p.pop() if v != u and v.lower() == u.lower() else p.append(u)

    return len(p) - 1


with open(input_for_day(5), "r") as input_file:
    data = input_file.read().strip()
min_length = min(
    collapse(s)
    for s in (
        data.replace(l, "").replace(l.swapcase(), "") for l in string.ascii_lowercase
    )
)
print(f"Answer: {min_length}")

Answer: 6118
CPU times: user 490 ms, sys: 0 ns, total: 490 ms
Wall time: 510 ms


## Day 6
### Part One
What is the size of the largest area that isn't infinite?

In [13]:
%%time
with open(input_for_day(6), "r") as input_file:
    points = np.fromiter(
        chain.from_iterable(map(int, l.strip().split(",")) for l in input_file), np.uint
    ).reshape(50, 2)

# Boundaries
tl = np.min(points, axis=0)
br = np.max(points, axis=0) + 1

# Index grid
xx, yy = np.mgrid[tl[0] : br[0], tl[1] : br[1]]
positions = np.vstack([xx.ravel(), yy.ravel()]).T

# Manhatten distances
dists = cdist(positions, points, metric="cityblock")
min_dists = np.min(dists, axis=1)

# Mapping to pointts
index_array = min_dists[..., np.newaxis] == dists

# Non ambigious distances
non_ambigious_index = np.sum(index_array, axis=1) == 1
index_array = index_array[non_ambigious_index]

# Borders to filter infinit regions
border_index = np.any(np.vstack([tl, br - 1])[:, np.newaxis] == positions, axis=(0, 2))
border_index = border_index[non_ambigious_index]
border_index = np.any(index_array[border_index], axis=0)

# Compute arrea for all points
area = np.sum(index_array, axis=0)

# Remove points that areas touch the borders
area[border_index] = -1

print(f"Answer: {np.max(area)}")

Answer: 4754
CPU times: user 78.8 ms, sys: 22.3 ms, total: 101 ms
Wall time: 122 ms


### Part Two
What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [14]:
%%time
with open(input_for_day(6), "r") as input_file:
    points = np.fromiter(
        chain.from_iterable(map(int, l.strip().split(",")) for l in input_file), np.uint
    ).reshape(50, 2)

# Boundaries
tl = np.min(points, axis=0)
br = np.max(points, axis=0) + 1

# Index grid
xx, yy = np.mgrid[tl[0] : br[0], tl[1] : br[1]]
positions = np.vstack([xx.ravel(), yy.ravel()]).T

# Manhatten distances
dists = cdist(positions, points, metric="cityblock")

print(f"Answer: {np.sum(np.sum(dists, axis=1) < 10000)}")

Answer: 42344
CPU times: user 20.6 ms, sys: 41.1 ms, total: 61.6 ms
Wall time: 78.7 ms


## Day 7
### Part One
In what order should the steps in your instructions be completed?

In [15]:
%%time
G = nx.DiGraph()
with open(input_for_day(7), "r") as input_file:
    G.add_edges_from(re.findall("\s(\w)\s", edge) for edge in input_file)

result = []
while G.nodes:
    result.append(min(G.in_degree(G.nodes), key=lambda x: (x[1], x[0]))[0])
    G.remove_node(result[-1])

print(f"Answer: {''.join(result)}")

Answer: JNOIKSYABEQRUVWXGTZFDMHLPC
CPU times: user 3.91 ms, sys: 1.04 ms, total: 4.95 ms
Wall time: 28.3 ms


### Part Two
With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

In [16]:
%%time
G = nx.DiGraph()
with open(input_for_day(7), "r") as input_file:
    G.add_edges_from(re.findall("\s(\w)\s", edge) for edge in input_file)

tick = 0
time_left = Counter({n: ord(n) - 4 for n in G.nodes})
while G.nodes:
    ready_nodes = [
        x[0]
        for x in sorted(G.in_degree(G.nodes), key=lambda x: (x[1], x[0]))
        if x[1] == 0
    ]
    nodes_changed = False
    while not nodes_changed:
        tick = tick + 1
        for node in ready_nodes[:5]:
            time_left.subtract(node)
            if time_left[node] == 0:
                G.remove_node(node)
                nodes_changed = True

print(f"Answer: {tick}")

Answer: 1099
CPU times: user 9.38 ms, sys: 0 ns, total: 9.38 ms
Wall time: 23.8 ms


## Day 8
### Part One
What is the sum of all metadata entries?

In [17]:
%%time
def sum_meta(tree):
    child_count = next(tree)
    metadata_count = next(tree)
    return sum(sum_meta(tree) for _ in range(child_count)) + sum(
        next(tree) for _ in range(metadata_count)
    )


with open(input_for_day(8), "r") as input_file:
    print(f"Answer: {sum_meta(map(int, input_file.read().split()))}")

Answer: 42798
CPU times: user 11.8 ms, sys: 370 µs, total: 12.1 ms
Wall time: 35.9 ms


### Part Two
What is the value of the root node?

In [18]:
%%time
def sum_childs(tree):
    child_count = next(tree)
    metadata_count = next(tree)
    child_values = [sum_childs(tree) for _ in range(child_count)]
    metadata_values = (next(tree) for _ in range(metadata_count))
    return (
        sum(child_values[i - 1] for i in metadata_values if 0 < i <= child_count)
        if child_count
        else sum(metadata_values)
    )


with open(input_for_day(8), "r") as input_file:
    print(f"Answer: {sum_childs(map(int, input_file.read().split()))}")

Answer: 23798
CPU times: user 12.8 ms, sys: 628 µs, total: 13.4 ms
Wall time: 31.2 ms
